In [1]:
#1) Написать программу, которая собирает входящие письма из своего или тестового почтового ящика и сложить 
# данные о письмах в базу данных (от кого, дата отправки, тема письма, текст письма полный)
#Логин тестового ящика: study.ai_172@mail.ru
#Пароль тестового ящика: NextPassword172!?

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from pymongo import MongoClient
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import hashlib
from random import getrandbits
from time import sleep

In [3]:
#Инициализация вебдрайвера
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(executable_path='./chromedriver', options=options)
driver.get('https://mail.ru/')

In [4]:
#Авторизация
wait = WebDriverWait(driver, 10)
name_wait = wait.until(EC.presence_of_element_located((By.NAME, 'login')))
login = driver.find_element_by_name('login')
login.send_keys('study.ai_172@mail.ru')
login.send_keys(Keys.ENTER)
sleep(.5)
wait = WebDriverWait(driver, 10)
name_wait = wait.until(EC.presence_of_element_located((By.NAME, 'password')))
password = driver.find_element_by_name('password')
password.send_keys('NextPassword172!?')
password.send_keys(Keys.ENTER)

In [5]:
#список сообщений
messages_link = set()

In [6]:
#заполним список ссылками
while True:
    flag = len(messages_link)
    try:
        wait = WebDriverWait(driver, 10)
        button_wait = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'dataset__items')))
    except: continue
    messages_block = driver.find_element_by_class_name('dataset__items')
    messages = messages_block.find_elements_by_tag_name('a')
    #добавим новые сообщения
    for message in messages:
        link = message.get_attribute('href')
        if (link is not None) and ('e.mail' in link):
            messages_link.add(link)
    #если не чего не добавилось то выходим
    if flag == len(messages_link): break
    # Прокрутим вниз
    actions = ActionChains(driver)
    actions.move_to_element(messages[-1])
    actions.perform()


In [7]:
len(messages_link)

912

In [8]:
#Словарь писем
info_emails = []
# обрабатываем письма
for link in messages_link:
    info_email = {}
    driver.get(link)
    try:
        wait = WebDriverWait(driver, 10)
        button_wait = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'letter__date')))
        info_email['date'] = driver.find_element_by_class_name('letter__date').text
        info_email['name'] = driver.find_element_by_class_name('letter-contact').text
        info_email['name_email'] = driver.find_element_by_class_name('letter-contact').get_attribute('title')
        info_email['topic'] = driver.find_element_by_tag_name('h2').text
        info_email['text'] = driver.find_element_by_class_name('letter__body').text
        info_email['_id'] =  hashlib.sha1(str(getrandbits(160)).encode()).hexdigest()
        info_emails.append(info_email)
    except: continue


In [9]:
#Сложим собранные данные в БД
client = MongoClient('localhost', 27017)
db = client['emails_database']
collections = db.messages
for item in info_emails:
    collections.update_one({ '_id': { '$eq':  item['_id']}}, {'$set': item}, upsert=True)

In [10]:
driver.close()

In [11]:
collections.delete_many({})